# Load Packages

In [1]:
import numpy as np
import warnings; warnings.filterwarnings('ignore')
import time
import sklearn.datasets
import numba
import scipy.linalg
import statsmodels.api

# Load toy dataset

In [2]:
tmp = sklearn.datasets.load_boston()
n_obs = len(tmp.target);

y = tmp.target
X = np.c_[ np.ones(shape=(n_obs, 1)), tmp.data[:,[5,12]] ]
#X = np.r_[ np.ones(shape=(1, n_obs)), tmp.data[:,[5,12]].T ]

In [3]:
print(y.shape, X.shape)
del tmp

(506,) (506, 3)


# Specify Linear Regression Solvers

In [4]:
fastmath = False
parallel = False

In [5]:
# LU decomposition
def fun1_numpy(y, X):
    return np.linalg.solve(np.dot(X.T, X), np.dot(X.T, y))

@numba.jit(nopython=True, fastmath=fastmath, parallel=parallel)
def fun1_jit(y, X):
    return np.linalg.solve(np.dot(X.T, X), np.dot(X.T, y))

# no numba
def fun1_scipy(y, X):  # calls LAPACK twice
    return scipy.linalg.lu_solve(scipy.linalg.lu_factor(np.dot(X.T,X)), np.dot(X.T,y))

@numba.jit(nopython=True, fastmath=fastmath, parallel=parallel)
def fun1_inv(y, X):  # linalg.inv is a wrapper for linalg.solve
    return np.dot(np.linalg.inv(np.dot(X.T, X)), np.dot(X.T,y)) 

In [6]:
# Pseudo-Inverse
def fun2_numpy(y, X, rcond=1e-15):
    return np.dot(np.linalg.pinv(np.dot(X.T, X), rcond=rcond), np.dot(X.T, y))

@numba.jit(nopython=True, fastmath=fastmath, parallel=parallel)
def fun2_jit(y, X, rcond=1e-15):
    return np.dot(np.linalg.pinv(np.dot(X.T, X), rcond=rcond), np.dot(X.T, y))

# no numba
def fun2_statsmodels(y, X, rcond=1e-15):
    return statsmodels.api.OLS(y,X).fit(method='pinv').params

In [7]:
# QR Factoring
def fun3_numpy(y, X):
    q, r = np.linalg.qr(np.dot(X.T, X))
    return np.dot(np.dot(np.linalg.inv(r), q.T), np.dot(X.T, y))

@numba.jit(nopython=True, fastmath=fastmath, parallel=parallel)
def fun3_jit(y, X):
    q, r = np.linalg.qr(np.dot(X.T, X))
    return np.dot(np.dot(np.linalg.inv(r), q.T), np.dot(X.T, y))

# no numba
def fun3_statsmodels(y, X, rcond=1e-15):
    return statsmodels.api.OLS(y,X).fit(method='qr').params

In [8]:
# SVD
def fun4_numpy(y, X, rcond=1e-15):
    beta, _, _, singu = np.linalg.lstsq(b=y, a=X, rcond=rcond)
    return beta

@numba.jit(nopython=True, fastmath=fastmath, parallel=parallel)
def fun4_jit(y, X, rcond=1e-15):
    beta, _, _, singu = np.linalg.lstsq(b=y, a=X, rcond=rcond)
    return beta

@numba.jit(nopython=True, fastmath=fastmath, parallel=parallel)
def fun4_svd(y, X, rcond=1e-15):
    u, s, v = np.linalg.svd(np.dot(X.T, X))
    diagtmp = np.eye(len(s)) * np.array([1.0/sigma  if sigma > rcond else 0.0 for sigma in s])
    return np.dot(np.dot(v, np.dot(diagtmp, u.T)), np.dot(X.T, y))

In [9]:
# Ridge Regression LU
# LU decomposition
def fun5_numpy(y, X, lam=0.01):
    n_vars = X.shape[1]
    P = np.dot(X.T, X) + lam * np.eye(n_vars)
    return np.linalg.solve(P, np.dot(X.T, y))

@numba.jit(nopython=True, fastmath=fastmath, parallel=parallel)
def fun5_jit(y, X, lam=0.01):
    n_vars = X.shape[1]
    P = np.dot(X.T, X) + lam * np.eye(n_vars)
    return np.linalg.solve(P, np.dot(X.T, y))

# Benchmark Execution Time

In [10]:
funcnames = [
    fun1_numpy, fun1_jit, fun1_scipy, fun1_inv,
    fun2_numpy, fun2_jit, fun2_statsmodels,
    fun3_numpy, fun3_jit, fun3_statsmodels,
    fun4_numpy, fun4_jit, fun4_svd,
    fun5_numpy, fun5_jit]

In [11]:
# Compile: Run it the first time
print('{0:6s} {1:6s} {2:s}'.format('Clock', 'CPU', 'function name'))
for func in funcnames:
    sh, sc = time.perf_counter(), time.process_time();
    beta = func(y,X); 
    #if beta is None: print('error solving')
    #beta = None;
    eh, ec = time.perf_counter(), time.process_time()
    print('{0:.4f} {1:.4f} {2:s}'.format(eh-sh, ec-sc, func.__name__))

Clock  CPU    function name
0.0003 0.0002 fun1_numpy
3.1646 2.7542 fun1_jit
0.0003 0.0004 fun1_scipy
0.5261 0.5122 fun1_inv
0.0005 0.0005 fun2_numpy
1.9254 1.8746 fun2_jit
0.0022 0.0016 fun2_statsmodels
0.0007 0.0010 fun3_numpy
1.5679 1.7466 fun3_jit
0.0022 0.0029 fun3_statsmodels
0.0002 0.0003 fun4_numpy
1.7580 1.9745 fun4_jit
1.2588 1.2480 fun4_svd
0.0004 0.0005 fun5_numpy
0.7740 0.7531 fun5_jit


In [12]:
n_trials = 50000
print('{0:7s} {1:7s} {2:s}'.format('Clock', 'CPU', 'function name'))
for func in funcnames:
    sh, sc = time.perf_counter(), time.process_time();
    for i in range(n_trials):
        beta = func(y,X); 
        #if beta is None: print('error solving')
        #beta = None;
    eh, ec = time.perf_counter(), time.process_time()
    print('{0:7.4f} {1:7.4f} {2:s}'.format(eh-sh, ec-sc, func.__name__))

Clock   CPU     function name
 1.6148  1.5480 fun1_numpy
 0.5349  0.5340 fun1_jit
 2.0732  2.0601 fun1_scipy
 0.7307  0.6387 fun1_inv
 5.2237  5.0372 fun2_numpy
 1.1127  1.0667 fun2_jit
28.0941 24.9214 fun2_statsmodels
 6.6487 13.0842 fun3_numpy
 1.2035  1.1715 fun3_jit
34.3556 59.3803 fun3_statsmodels
 3.6985  3.8746 fun4_numpy
 9.1950  8.2872 fun4_jit
 8.1461  7.4237 fun4_svd
 2.3727  2.2830 fun5_numpy
 6.3682  6.0540 fun5_jit
